**Install libraries**

In [ ]:
!pip install einops
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation
!pip install accelerate
!pip install optimum

In [ ]:
!pip install datasets

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers import (
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
)

# Inference with [Optimization](https://huggingface.co/docs/transformers/en/perf_infer_gpu_one?install=NVIDIA)

In [6]:
import torch
# load model in 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from tqdm import tqdm

**Flash Attention compatibility**

In [7]:
import os
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
 os.system("pip install flash-attn")
 torch_dtype = torch.bfloat16
 attn_implementation='flash_attention_2'
 print("Your GPU is compatible with FlashAttention and bfloat16.")
else:
 torch_dtype = torch.float16
 attn_implementation='eager'
 print("Your GPU is not compatible with FlashAttention and bfloat16.")

Your GPU is not compatible with FlashAttention and bfloat16.


In [8]:

# Load model directly
# from transformers import AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained("internlm/internlm2-7b", trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained("internlm/internlm2-20b", trust_remote_code=True)
# Set `torch_dtype=torch.float16` to load model in float16, otherwise it will be loaded as float32 and might cause OOM Error.
model = AutoModelForCausalLM.from_pretrained("internlm/internlm2-20b",quantization_config=quantization_config, device_map={"": 0},  attn_implementation=attn_implementation)#.cuda()
#trust_remote_code=True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

tokenization_internlm2_fast.py:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

tokenization_internlm2.py:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-20b:
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-20b:
- tokenization_internlm2_fast.py
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/9.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

**Currently supported models for BetterTransformer**

'albert', 'bark', 'bart', 'bert', 'bert-generation', 'blenderbot', 'bloom', 'camembert', 'blip-2', 'clip', 'codegen', 'data2vec-text', 'deit', 'distilbert', 'electra', 'ernie', 'fsmt', 'gpt2', 'gptj', 'gpt_neo', 'gpt_neox', 'hubert', 'layoutlm', 'm2m_100', 'marian', 'markuplm', 'mbart', 'opt', 'pegasus', 'rembert', 'prophetnet', 'roberta', 'roc_bert', 'roformer', 'splinter', 'tapas', 't5', 'vilt', 'vit', 'vit_mae', 'vit_msn', 'wav2vec2', 'xlm-roberta', 'yolos']

In [ ]:
# enable BetterTransformer - NOT WORKING/SUPPORTED for the given model
model_bt = model.to_bettertransformer()

In [10]:
input_text = "Hello my dog is cute and"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

In [ ]:
model_cuda = model.to("cuda")
#`.to` is not supported for `4-bit` or `8-bit` bitsandbytes models.
#Please use the model as it is, since the model has already been set to the
#correct devices and casted to the correct `dtype`.

In [12]:
gen_kwargs = {"max_length": 128, "top_p": 0.8, "temperature": 0.8, "do_sample": True, "repetition_penalty": 1.0}
output = model.generate(**inputs, **gen_kwargs)
output = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
print(output)

Hello my dog is cute and I need a dog sitter for 3 days a week. My dog is a 1 year old 90 pound labradoodle. He loves to play with other dogs, run around, and go for walks. I also have a cat who is very friendly. We also have a pool so he can swim in it. We are looking for someone to come to our house in the mornings to let the dogs out, feed them, and play with them. Then in the afternoon you will come back to let them out and play with them. We will pay you $100 for the 3 days


**Text Classification**

In [14]:
prompt = ["Predict the sentiment of the input. Answer either {positive/negative/neutral}. Input: I Love you.Answer"]
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
# for k,v in inputs.items():
#     inputs[k] = v.cuda()
# gen_kwargs = {"max_length": 128, "top_p": 0.8, "temperature": 0.8, "do_sample": True, "repetition_penalty": 1.0}
output = model.generate(**inputs,max_length=100) #**gen_kwargs)
output = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
print(output)

Predict the sentiment of the input. Answer either {positive/negative/neutral}. Input: I Love you.Answer: positive


**On Complete dataset** - [FPB](https://huggingface.co/datasets/financial_phrasebank)

In [16]:
from datasets import load_dataset

fpb_dataset = load_dataset("financial_phrasebank",'sentences_allagree')

**Format the dataset as a prompt**

In [43]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# tokenizer.to(device)

In [29]:
labels = "positive/negative/neutral"
p=[]
for i in range(len(fpb_dataset['train']['sentence'])):
  p.append(f"Predict the sentiment of the input. Answer either {labels}. Input: {fpb_dataset['train']['sentence'][i]}. Answer:")

fpb_dataset['train'] = fpb_dataset['train'].add_column("prompt_2", p)

In [44]:
fpb_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'prompt', 'prompt_2'],
        num_rows: 2264
    })
})

In [45]:
# tokenized_set_1 = flant5_tokenizer(prompt, return_tensors='pt', padding=True,max_length=512)

#---------------------------------------------------------

tokenized_set = tokenizer.batch_encode_plus(
    fpb_dataset['train']['prompt_2'], #prompt
    return_tensors='pt',
    add_special_tokens=True,
    return_attention_mask=True,
    # pad_to_max_length=True,  Deprecated
    padding=True,
    truncation=True,
    max_length=175 # the maximum lenght observed in the headlines, as can be seen in the above distribution
)

In [46]:
input_ids_val = tokenized_set['input_ids']
attention_masks_val = tokenized_set['attention_mask']

dataset_val = TensorDataset(input_ids_val, attention_masks_val)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=5)

In [47]:
import time, torch
predictions = []
start_time = time.time()
model.eval()

for batch in dataloader_validation:

  batch = tuple(b.to(device) for b in batch)

  inputs = {'input_ids':batch[0],
                  'attention_mask': batch[1],
                 }

  with torch.no_grad():
    outputs = model.generate(**inputs, max_length=200)
    # outputs = flant5_model(**inputs)

  res_sentences = [tokenizer.decode(i) for i in outputs]
  # out_text = [o.split("Answer:")[1] for o in res_sentences]

  # # show results
  # for gen_text in out_text:
  #     print(gen_text)

  end_time = time.time()
  execution_time = end_time - start_time

  predictions.append(res_sentences)
# print("Result:", res_sentences)
print("Execution time:", execution_time, "seconds")

Execution time: 1211.177314043045 seconds


**Extract response from models output**

In [49]:
final_sent_answer=[]
for batch in predictions:
  for sent_output in batch:
    final_sent_answer.append(sent_output.split('Answer:')[1].split('<')[0])

In [54]:
for index,res in enumerate(final_sent_answer):
  final_sent_answer[index] = res.replace(" ","")

In [56]:
len(final_sent_answer)

2264

In [57]:
# prompt: convert list to dataframe with column called 'response'

import pandas as pd
df_res = pd.DataFrame(final_sent_answer, columns=['response'])


In [62]:
df_res['label'] = [1] *2264

In [69]:
df_res.tail(5)

,response,label
2259,negative,0
2260,negative,0
2261,negative,0
2262,negative,0
2263,negative,0


In [ ]:
for index,res in enumerate(df_res['response']):
  # print(index)
  if df_res['response'][index] == 'positive':
    df_res['label'][index] = 2
  elif df_res['response'][index] == 'negative':
    df_res['label'][index] = 0
  elif df_res['response'][index] == 'neutral':
    df_res['label'][index] = 1


In [70]:
import pandas as pd
df_res.to_csv('final_sent_answer_Internlm-20b.csv')

# **Evaluate Model**

In [71]:
# prompt: code to convert dataset object fpb_dataset to pandas dataframe object

fpb_dataset_pandas = fpb_dataset['train'].to_pandas()
# Compare the values in the two columns
comparison = fpb_dataset_pandas["label"] == df_res["label"]

# Calculate the accuracy
accuracy = comparison.sum() / len(comparison)

# Print the accuracy
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.90


In [72]:
# prompt: code to calculate precision

from sklearn.metrics import precision_score

y_true = fpb_dataset['train']["label"]
y_pred = df_res["label"]

precision = precision_score(y_true, y_pred, average='macro')
print("Macro-averaged precision:", precision)

Macro-averaged precision: 0.9085760892778437


In [73]:
# prompt: code to calculate f1 score

from sklearn.metrics import f1_score

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='macro')

# Print F1 score
print("Macro-averaged F1 score:", f1)


Macro-averaged F1 score: 0.8875098950103918


In [74]:
# prompt: code to calculate recall

from sklearn.metrics import recall_score

# Calculate recall score
recall = recall_score(y_true, y_pred, average='macro')

# Print recall score
print("Macro-averaged recall:", recall)


Macro-averaged recall: 0.8704838348680304
